In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle

In [3]:
## Load the dataset

df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Drop irrevalent Columns...
df = df.iloc[: , 3:]

In [5]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## Encode the Categorical Variable

le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])
df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
## OneHotEnoder on Geography columns

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output= False , drop='first')
geo_encoder = ohe.fit_transform(df[['Geography']])
geo_encoder

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       ...,
       [0., 0.],
       [1., 0.],
       [0., 0.]], shape=(10000, 2))

In [8]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_Germany', 'Geography_Spain'], dtype=object)

In [9]:
geo_encoded_df = pd.DataFrame(geo_encoder ,columns=ohe.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_Germany,Geography_Spain
0,0.0,0.0
1,0.0,1.0
2,0.0,0.0
3,0.0,0.0
4,0.0,1.0
...,...,...
9995,0.0,0.0
9996,0.0,0.0
9997,0.0,0.0
9998,1.0,0.0


In [10]:
## Combine all the Columns with the original data

df = pd.concat([df.drop('Geography', axis = 1) , geo_encoded_df] , axis =1)

In [11]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,1.0,0.0


In [13]:
## Split data into independent and independent Features
X = df.drop('EstimatedSalary', axis=1)
y = df['EstimatedSalary']

In [14]:
## Split into Train and Test Data
X_train , X_test , y_train , y_test = train_test_split(X,y , test_size=0.2 , random_state= 42)

## Scaling --> Standard Sclar
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# Save the encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(le_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(ohe, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [16]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0


## Train ANN with Regression Problem Statement

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [19]:
## Build Our ANN Models...
model = Sequential([
    Dense(64 , activation='relu' , input_shape= (X_train.shape[1] ,)),  ##  HL1 --> Connected with input layer 
    Dense(32 , activation='relu'),  ##  HL2...
    Dense(1) ##  Output Layer...   # Default activation --> linear
])

## Compile the Model
model.compile(optimizer='adam' , loss="mean_absolute_error" , metrics=['mae'])

model.summary()

d:\All ML PROJECT\ANN End to End Project\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

## Set up the Tensorboard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir , histogram_freq= 1) 

In [21]:
## Setup Early Stopping...
early_stopping_callback = EarlyStopping(monitor='val_loss' , patience=10 , restore_best_weights=True)


In [22]:
## Training the model
history = model.fit(
    X_train , y_train , validation_data=(X_test , y_test), epochs = 100,
    callbacks= [tensorflow_callback , early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 100370.9141 - mae: 100370.9141 - val_loss: 98489.3125 - val_mae: 98489.3125
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99526.2656 - mae: 99526.2656 - val_loss: 96796.6719 - val_mae: 96796.6719
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 96574.8203 - mae: 96574.8203 - val_loss: 92482.3672 - val_mae: 92482.3672
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 90824.0781 - mae: 90824.0781 - val_loss: 85408.0938 - val_mae: 85408.0938
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 82594.8750 - mae: 82594.8750 - val_loss: 76444.4922 - val_mae: 76444.4922
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 73100.1562 - mae: 73100.1562 - val_loss: 67363.3516 - val_mae: 67363.3516
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 64309.3477 - mae: 64309.3477 - val_loss: 59834.5898 - val_mae: 59834.5898
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 

In [40]:
 ## Load the Tensorboard Extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# %tensorboard --logdir logs/fit
# %tensorboard --logdir=tensorboard_logs
%tensorboard --logdir=logs/fit  

Reusing TensorBoard on port 6008 (pid 177264), started 0:00:01 ago. (Use '!kill 177264' to kill it.)

In [44]:
## Evaluate the model on Test data
test_loss , test_mae = model.evaluate(X_test , y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50267.2383 - mae: 50267.2383
Test MAE : 50267.23828125


In [45]:
model.save('regression_model.h5')